<a href="https://colab.research.google.com/github/MiguelMendozaG/CANproject/blob/6_features/LSTM_Autoencoder_Car.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from sklearn.model_selection import train_test_split
from keras import optimizers
DATA_SPLIT_PCT = 0.2
SEED = 123
max_size = 100
n_features = 6
lookback = max_size

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
model_dir = '/content/gdrive/My Drive/'

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
car_model = "Colab Notebooks/Car"

In [ ]:
model_dir + car_model

'/content/gdrive/My Drive/Colab Notebooks/Car'

In [ ]:
data = np.load(model_dir + car_model + '/recorrido.npy')

In [ ]:
def temporalize(X, y, lookback):
    output_X = []
    output_y = []
    for i in range(len(X)-lookback-1):
        t = []
        for j in range(0,lookback):
            # Gather past records upto the lookback period
            t.append(X[[(i+j)], :])
        output_X.append(t)
        output_y.append(y[i+lookback])
    return output_X, output_y


In [ ]:
def flatten(X):
    '''
    Flatten a 3D array.
    
    Input
    X            A 3D array for lstm, where the array is sample x timesteps x features.
    
    Output
    flattened_X  A 2D array, sample x features.
    '''
    flattened_X = np.empty((X.shape[0], X.shape[2]) )  # sample x features array.
    for i in range(X.shape[0]):
      flattened_X[i] = X[i, (X.shape[1]-1), :]
    return(flattened_X)

def scale(X, scaler):
    '''
    Scale 3D array.

    Inputs
    X            A 3D array for lstm, where the array is sample x timesteps x features.
    scaler       A scaler object, e.g., sklearn.preprocessing.StandardScaler, sklearn.preprocessing.normalize
    
    Output
    X            Scaled 3D array.
    '''
    for i in range(X.shape[0]):
        X[i, :, :] = scaler.transform(X[i, :, :])
        
    return X

In [ ]:
X_train, X_test = train_test_split(np.array(data), test_size=DATA_SPLIT_PCT, random_state=SEED)
X_train, X_validation = train_test_split(X_train, test_size=DATA_SPLIT_PCT, random_state=SEED)
X_train_original_shape = X_train.shape
X_test_original_shape = X_test.shape
X_validation_original_shape = X_validation.shape

In [ ]:
X_train_reshape = X_train.reshape(X_train.shape[0], lookback, n_features)
print (X_train_reshape.shape)
print (X_train_reshape[:,:,0])

(24788, 100, 6)
[[9223440897264410624 9223440622411669504 9223440622411669504 ...
  9223440759846428672 9223440484989493248 9223440484989493248]
 [9223444126832349184 9223444126832349184 9223444126832349184 ...
  9223439661274324992 9223439661274324992 9223439798822330368]
 [9223440484893011968 9223440484893011968 9223440759741558784 ...
  9223440210040270848 9223440347462447104 9223440347462447104]
 ...
 [9223440347420499968 9223440347420499968 9223440347420499968 ...
  9223440622277435392 9223440622277435392 9223440209998323712]
 [9223440347558932480 9223440622403284992 9223440622403284992 ...
  9223440347533766656 9223440347563126784 9223440347563126784]
 [9223446052500238336 9223445915128393728 9223445915128393728 ...
  9223445091513892864 9223445091513892864 9223445091513892864]]


In [ ]:
X_train_split = flatten(X_train_reshape)
X_train_array = np.array(np.hsplit( X_train_split, 6))

In [ ]:
# define min max scalers for each message
scaler1 = MinMaxScaler()
scaler2 = MinMaxScaler()
scaler3 = MinMaxScaler()
scaler4 = MinMaxScaler()
scaler5 = MinMaxScaler()
scaler6 = MinMaxScaler()

X_train_array_1 = scaler1.fit(X_train_reshape[:,:,0])
X_train_array_2 = scaler2.fit_transform(X_train_reshape[:,:,1])
X_train_array_3 = scaler3.fit_transform(X_train_reshape[:,:,2])
X_train_array_4 = scaler4.fit_transform(X_train_reshape[:,:,3])
X_train_array_5 = scaler5.fit_transform(X_train_reshape[:,:,4])
X_train_array_6 = scaler6.fit_transform(X_train_reshape[:,:,5])

X_array_float = np.array()

In [ ]:
print (scaler1)
print (X_train_reshape[0:2,:,0].shape)
a = scaler1.transform(X_train_reshape[0:2,:,0])
print (a.shape)
print (a)


MinMaxScaler(copy=True, feature_range=(0, 1))
(2, 100)
(2, 100)
[[0.59090861 0.59090844 0.59090844 0.68420954 0.59090844 0.59090844
  0.59090844 0.59090829 0.59090835 0.59090835 0.59090835 0.59090835
  0.59090835 0.59090835 0.684208   0.59090835 0.59090852 0.59090852
  0.59090852 0.6842083  0.59090861 0.59090861 0.59090861 0.59090844
  0.59090844 0.59090844 0.59090826 0.59090826 0.59090826 0.59090826
  0.59090852 0.59090852 0.6842082  0.59090852 0.59090896 0.59090904
  0.59090904 0.59090887 0.59090887 0.59090887 0.59090887 0.59090835
  0.59090835 0.59090835 0.59090803 0.59090818 0.59090818 0.59090818
  0.59090852 0.6842082  0.59090852 0.5909087  0.5909087  0.5909087
  0.5909087  0.6842082  0.59090852 0.59090852 0.59090852 0.59090826
  0.59090826 0.59090826 0.59090809 0.59090809 0.59090809 0.6842077
  0.59090818 0.59090818 0.59090818 0.59090844 0.59090844 0.59090844
  0.59090844 0.59090861 0.59090861 0.59090861 0.59090861 0.59090861
  0.59090861 0.59090861 0.59090852 0.59090852 0.590908

In [ ]:
print (X_train_reshape[0,0:2,0])
print(type(a[0,0]))
#X_train_reshape[0,0,0] = a[0,0]
print (type(X_train_reshape[0,0,0]))

[9223440897264410624 9223440622411669504]
<class 'numpy.float64'>
<class 'numpy.uint64'>


In [ ]:
#Train statistics
print ("Msg 1")
print(X_train_array_1)
print (X_train_array_1.min(), X_train_array_1.max())
print ("Msg 2")
print(X_train_array_2)
print (X_train_array_2.min(), X_train_array_2.max())
print ("Msg 3")
print(X_train_array_3)
print (X_train_array_3.min(), X_train_array_3.max())
print ("Msg 4")
print(X_train_array_4)
print (X_train_array_4.min(), X_train_array_4.max())
print ("Msg 5")
print(X_train_array_5)
print (X_train_array_5.min(), X_train_array_5.max())
print ("Msg 6")
print(X_train_array_6)
print (X_train_array_6.min(), X_train_array_6.max())

Msg 1
[[0.59090861 0.59090844 0.59090844 ... 0.59090852 0.68420848 0.684208  ]
 [0.59091065 0.59091065 0.59091065 ... 0.59090783 0.68420788 0.6842075 ]
 [0.59090835 0.59090835 0.59090852 ... 0.59090818 0.68420838 0.6842079 ]
 ...
 [0.59090826 0.59090826 0.59090826 ... 0.59090844 0.68420858 0.6842078 ]
 [0.59090826 0.59090844 0.59090844 ... 0.59090826 0.68420838 0.6842079 ]
 [0.59091186 0.59091177 0.59091177 ... 0.59091125 0.68421184 0.68421136]]
0.0 1.0
Msg 2
[[0.2686968  0.24653613 0.24929835 ... 0.25761646 0.23545579 0.23545579]
 [0.51246369 0.51246369 0.51820541 ... 0.14681361 0.14681361 0.10249244]
 [0.19113411 0.19113411 0.21568456 ... 0.16897344 0.18005377 0.18005377]
 ...
 [0.19113428 0.19113428 0.19327577 ... 0.20221461 0.20221461 0.1689736 ]
 [0.22437545 0.25761646 0.26050283 ... 0.24653613 0.22437545 0.22437545]
 [0.67867008 0.66758975 0.67506952 ... 0.62049916 0.62049916 0.62049916]]
0.0 1.0
Msg 3
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.0

In [ ]:
#split test and validation
X_test_reshape = X_test.reshape(X_test.shape[0], lookback, n_features)
X_test_flatten = flatten(X_test_reshape)
X_test_array = np.array(np.hsplit( X_test_flatten, 6))
print (X_test_array.shape)

X_validation_reshape = X_validation.reshape(X_validation.shape[0], lookback, n_features)
X_validation_flatten = flatten(X_validation_reshape)
X_validation_array = np.array(np.hsplit( X_validation_flatten, 6))
print (X_validation_array.shape)

(6, 7747, 1)
(6, 6198, 1)


In [ ]:
#transform test and validation with minmax statistics
X_test_array_1 = scaler1.transform(X_test_array[0])
X_test_array_2 = scaler2.transform(X_test_array[1])
X_test_array_3 = scaler3.transform(X_test_array[2])
X_test_array_4 = scaler4.transform(X_test_array[3])
X_test_array_5 = scaler5.transform(X_test_array[4])
X_test_array_6 = scaler6.transform(X_test_array[5])

X_validation_array_1 = scaler1.transform(X_validation_array[0])
X_validation_array_2 = scaler2.transform(X_validation_array[1])
X_validation_array_3 = scaler3.transform(X_validation_array[2])
X_validation_array_4 = scaler4.transform(X_validation_array[3])
X_validation_array_5 = scaler5.transform(X_validation_array[4])
X_validation_array_6 = scaler6.transform(X_validation_array[5])

In [ ]:
#Test statistics
print ("Msg 1")
print(X_test_array_1)
print (X_test_array_1.min(), X_test_array_1.max())
print ("Msg 2")
print(X_test_array_2)
print (X_test_array_2.min(), X_test_array_2.max())
print ("Msg 3")
print(X_test_array_3)
print (X_test_array_3.min(), X_test_array_3.max())
print ("Msg 4")
print(X_test_array_4)
print (X_test_array_4.min(), X_test_array_4.max())
print ("Msg 5")
print(X_test_array_5)
print (X_test_array_5.min(), X_test_array_5.max())
print ("Msg 6")
print(X_test_array_6)
print (X_test_array_6.min(), X_test_array_6.max())

Msg 1
[[0.68421071]
 [0.6842083 ]
 [0.6842081 ]
 ...
 [0.6842081 ]
 [0.6842081 ]
 [0.6842081 ]]
0.15789527238881096 1.1578919099500897
Msg 2
[[0.52077563]
 [0.24376587]
 [0.2216052 ]
 ...
 [0.26869731]
 [0.24653613]
 [0.23268554]]
1.6907251154663072e-07 0.9889200019888209
Msg 3
[[0.99604743]
 [0.        ]
 [0.        ]
 ...
 [0.47430636]
 [0.        ]
 [0.        ]]
0.0 0.9999999999705366
Msg 4
[[0.51960074]
 [0.04524816]
 [0.04524816]
 ...
 [0.04403853]
 [0.49741665]
 [0.04524816]]
0.0 0.9999991575636475
Msg 5
[[0.74972544]
 [0.84356315]
 [0.84356315]
 ...
 [0.84368517]
 [0.84368517]
 [0.8438072 ]]
0.0 1.0
Msg 6
[[9.96031746e-01]
 [3.04606428e-15]
 [3.04606428e-15]
 ...
 [4.68253968e-01]
 [4.84621162e-16]
 [7.07546896e-16]]
0.0 0.9999999999999992


In [ ]:
#Validation statistics
print ("Msg 1")
print(X_validation_array_1)
print (X_validation_array_1.min(), X_validation_array_1.max())
print ("Msg 2")
print(X_validation_array_2)
print (X_validation_array_2.min(), X_validation_array_2.max())
print ("Msg 3")
print(X_validation_array_3)
print (X_validation_array_3.min(), X_validation_array_3.max())
print ("Msg 4")
print(X_validation_array_4)
print (X_validation_array_4.min(), X_validation_array_4.max())
print ("Msg 5")
print(X_validation_array_5)
print (X_validation_array_5.min(), X_validation_array_5.max())
print ("Msg 6")
print(X_validation_array_6)
print (X_validation_array_6.min(), X_validation_array_6.max())

Msg 1
[[0.6842081 ]
 [0.68421266]
 [0.6842078 ]
 ...
 [0.68420739]
 [0.6842078 ]
 [0.6842086 ]]
0.0 1.1578919099500897
Msg 2
[[0.23268554]
 [0.74515159]
 [0.21329512]
 ...
 [0.14404437]
 [0.21329512]
 [0.33517932]]
0.0 0.9778393276875939
Msg 3
[[0.        ]
 [0.30830079]
 [0.        ]
 ...
 [0.        ]
 [0.        ]
 [0.        ]]
0.0 1.0000002411574607
Msg 4
[[4.52481553e-02]
 [3.07368804e-01]
 [4.97416650e-01]
 ...
 [4.52634743e-02]
 [4.97416650e-01]
 [2.33750375e-10]]
0.0 0.9999220173685515
Msg 5
[[0.8438072 ]
 [0.74996949]
 [0.8438072 ]
 ...
 [0.75009152]
 [0.84368517]
 [0.84368517]]
0.0 0.9998779743746187
Msg 6
[[1.64859308e-15]
 [3.07539683e-01]
 [7.07546896e-16]
 ...
 [1.64859308e-15]
 [1.40452024e-15]
 [2.43720388e-15]]
0.0 0.9999999999999984


In [ ]:
'''
print(X_test.shape)
#X_test_1 = Scaler(X_test, mm_scaler)
X_test = mm_scaler.transform(X_test.reshape(X_test.shape[0]*X_test.shape[1], 1))
X_validation = mm_scaler.transform(X_validation.reshape(X_validation.shape[0]*X_validation.shape[1], 1))
print (X_test.shape)
print (X_validation.shape)
X_test = X_test.reshape(X_test_original_shape[0], max_size, 1)
X_validation = X_validation.reshape(X_validation_original_shape[0], max_size, 1)
print (X_test)'''

(7747, 100, 1)
(774700, 1)
(619800, 1)
[[[0.59090844]
  [0.5909087 ]
  [0.5909087 ]
  ...
  [0.59090835]
  [0.59090826]
  [0.59090826]]

 [[0.59091117]
  [0.59091117]
  [0.59091117]
  ...
  [0.59091151]
  [0.5909116 ]
  [0.5909116 ]]

 [[0.59091052]
  [0.59091052]
  [0.59091052]
  ...
  [0.59091008]
  [0.59091008]
  [0.59091008]]

 ...

 [[0.59090826]
  [0.59090826]
  [0.59090826]
  ...
  [0.59090818]
  [0.59090818]
  [0.59090818]]

 [[0.59090826]
  [0.59090826]
  [0.59090826]
  ...
  [0.59090826]
  [0.59090826]
  [0.59090826]]

 [[0.59090861]
  [0.59090878]
  [0.59090887]
  ...
  [0.59090835]
  [0.59090835]
  [0.59090835]]]


In [ ]:
X_train = X_train.reshape(X_train_original_shape[0], max_size, 1)

In [ ]:
timesteps = X_train.shape[1]
n_features = X_train.shape[2]
lr = 0.0001
print(timesteps)
print(n_features)

100
1


In [ ]:
lstm_autoencoder = Sequential()
# Encoder
lstm_autoencoder.add(LSTM(32, activation='relu', input_shape=(timesteps, n_features), return_sequences=True))
lstm_autoencoder.add(LSTM(16, activation='relu', return_sequences=False))
lstm_autoencoder.add(RepeatVector(timesteps))
# Decoder
lstm_autoencoder.add(LSTM(16, activation='relu', return_sequences=True))
lstm_autoencoder.add(LSTM(32, activation='relu', return_sequences=True))
lstm_autoencoder.add(TimeDistributed(Dense(n_features)))

lstm_autoencoder.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 32)           4352      
_________________________________________________________________
lstm_1 (LSTM)                (None, 16)                3136      
_________________________________________________________________
repeat_vector (RepeatVector) (None, 100, 16)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 16)           2112      
_________________________________________________________________
lstm_3 (LSTM)                (None, 100, 32)           6272      
_________________________________________________________________
time_distributed (TimeDistri (None, 100, 1)            33        
Total params: 15,905
Trainable params: 15,905
Non-trainable params: 0
____________________________________________________

In [ ]:
adam = optimizers.Adam(lr)
lstm_autoencoder.compile(loss='mse', optimizer=adam)
# fit model
lstm_autoencoder_history = lstm_autoencoder.fit(X_train, X_train, validation_data=(X_validation, X_validation),
                                                verbose=1, epochs=300, batch_size=100).history
yhat = lstm_autoencoder.predict(X_train, verbose=0)

plt.plot(lstm_autoencoder_history['loss'], linewidth=2, label='Train')
plt.plot(lstm_autoencoder_history['val_loss'], linewidth=2, label='Valid')
plt.legend(loc='upper right')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()

Epoch 1/300
248/248 [==============================] - 143s 578ms/step - loss: 0.1311 - val_loss: 0.0216
Epoch 2/300
248/248 [==============================] - 137s 552ms/step - loss: 0.0150 - val_loss: 0.0085
Epoch 3/300
248/248 [==============================] - 133s 536ms/step - loss: 0.0083 - val_loss: 0.0076
Epoch 4/300
248/248 [==============================] - 129s 520ms/step - loss: 0.0050 - val_loss: 5.7844e-04
Epoch 5/300
 84/248 [=========>....................] - ETA: 1:21 - loss: 4.4112e-04

KeyboardInterrupt: ignored